In [1]:
import pickle as pickle
import pandas as pd
import re

In [2]:
df = pd.read_pickle("final_data.pkl")
df.head()

,uri,track,artist,cover image,lyrics
0,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,Mercury: Retrograde,ghostemane,https://i.scdn.co/image/ab67616d0000b27347fa48...,"[Intro]Ask me if I give a fuck about a clique,..."
1,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,Pathology,don kenobi,https://i.scdn.co/image/ab67616d0000b27391b3f8...,[Intro: Don Kenobi]In due time it feels like t...
2,spotify:track:0vSWgAlfpye0WCGeNmuNhy,Symbiote,gizmo,https://i.scdn.co/image/ab67616d0000b2731981f3...,"[Verse]Aye, AyeSmoke a Dutch bluntDon't flick ..."
4,spotify:track:4RYhSQFFlquCTb8XRMJgFU,Venom,$uicideboy$,https://i.scdn.co/image/ab67616d0000b2737811bf...,"[Intro: D.J. Live Wire]Ratatatatatata, boom!Ra..."
5,spotify:track:6fsypiJHyWmeINsOLC1cos,Gatteka,ghostemane,https://i.scdn.co/image/ab67616d0000b273b5ea1c...,"[Intro]Ayy, ayy, uh, uh, uh[Chorus]See, I just..."


In [3]:
def clean_text(x):
    # remove tags such as [Intro] and [Chorus]
    cleaned_str = re.sub(r'\[.*?\]', '', x)
    # fix typos by adding space before capitalized letter 
    cleaned_str = re.sub(r"(?<=\w)([A-Z])", r". \1", cleaned_str)
    # seperate paren lyrics
    cleaned_str = re.sub(r"(\S)\)", r"\1). ", cleaned_str)
    return cleaned_str

In [4]:
df['lyrics_cleaned'] = df['lyrics'].apply(lambda x: clean_text(x))
df.head()

,uri,track,artist,cover image,lyrics,lyrics_cleaned
0,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,Mercury: Retrograde,ghostemane,https://i.scdn.co/image/ab67616d0000b27347fa48...,"[Intro]Ask me if I give a fuck about a clique,...","Ask me if I give a fuck about a clique, ayy. A..."
1,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,Pathology,don kenobi,https://i.scdn.co/image/ab67616d0000b27391b3f8...,[Intro: Don Kenobi]In due time it feels like t...,In due time it feels like this all makes sense...
2,spotify:track:0vSWgAlfpye0WCGeNmuNhy,Symbiote,gizmo,https://i.scdn.co/image/ab67616d0000b2731981f3...,"[Verse]Aye, AyeSmoke a Dutch bluntDon't flick ...","Aye, Aye. Smoke a Dutch blunt. Don't flick the..."
4,spotify:track:4RYhSQFFlquCTb8XRMJgFU,Venom,$uicideboy$,https://i.scdn.co/image/ab67616d0000b2737811bf...,"[Intro: D.J. Live Wire]Ratatatatatata, boom!Ra...","Ratatatatatata, boom!Ratatatatatata, boom!Two ..."
5,spotify:track:6fsypiJHyWmeINsOLC1cos,Gatteka,ghostemane,https://i.scdn.co/image/ab67616d0000b273b5ea1c...,"[Intro]Ayy, ayy, uh, uh, uh[Chorus]See, I just...","Ayy, ayy, uh, uh, uh. See, I just can’t relate..."


In [5]:
test = df.loc[0, 'lyrics_cleaned']
print(test)

Ask me if I give a fuck about a clique, ayy. Ask me if I give a fuck about a diss, yuh. Ask me if I give a fuck about fame, yuh. Recently I just don't give a fuck about a thing, yuh, ayy. Ask me if I give a fuck about a diss, ayy. Ask me if I give a fuck about a clique, yuh. Ask me if I give a fuck about fame, yuh. Recently I just don't give a fuck about a thing, yuh. Ayy, I just decided by the grace of the god Poseidon. That you're so dead to me I dug a hole for you to lie in. I'm sick and disowning, all the moments, and the key components. That led me to follow hollow promises from empty monads. It's 11 degrees and I can see my breath so I know I'm breathing. But I got no pulse, I swear to Thelema my heart ain't beating. I better get back to the black hole sun. Leaving my gun, I don't need it for this one. I'm finding the silver lining and I'm mining for hope. Tryna keep my wrists closed. You are toxic. My blood, your lips. You are toxic. My blood, your lips


In [6]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import Counter
import pandas as pd

In [7]:
# source: https://github.com/anasbadawy/Song-Lyrics-Compilation-based-on-Machine-Learning/blob/master/final.py

# cleaning Text
def clearText(String):
    pattern = re.compile(r'\b(' + r'|'.join(
        stopwords.words('english')) + r')\b\s*')  # \b: Matches only the beginning or end of the word.
    text = re.sub(r'\W+', ' ', String)  # for symbol
    text = re.sub(r'\d+', ' ', text)  # digits
    text = pattern.sub(' ', text.lower())
    text = re.sub(r'\s+', ' ', text)  # for space
    tokenize = word_tokenize(text)
    return (tokenize)

def summarizedSong(lyrics, n):
    tokenize = clearText(lyrics)

    fdist = FreqDist(tokenize)
    max = fdist.most_common(1)

    # convert to weight
    for x in fdist:
        fdist[x] = round(fdist[x] / max[0][1], 2)

    # Product sentence scores
    sent2score = {}
    sent2score = {sentence: fdist[x] if sentence not in sent2score.keys() else sent2score[sentence] + fdist[x]
                  for sentence in re.split('[?.!]', lyrics) for x in sentence.split(' ') if x in fdist.keys()}
    top_common = dict(Counter(sent2score).most_common(n)).keys()
    return ([*top_common])

In [8]:
df['extract_summary'] = df['lyrics_cleaned'].apply(lambda x: ".".join(summarizedSong(x, 2)).strip())
df['extract_summary'][0]

'Ask me if I give a fuck about a diss, yuh. Ask me if I give a fuck about fame, yuh'